# Shor's algorithm for integer factoring

This example implementation of Shor's algorithm is [adapted from ProjectQ](https://github.com/ProjectQ-Framework/ProjectQ/blob/develop/examples/shor.py).

In [1]:
import math
import random
from fractions import Fraction

try:
    from math import gcd
except ImportError:
    from fractions import gcd

from pyqrack import QrackSimulator, Pauli

import os
os.environ["QRACK_QUNIT_SEPARABILITY_THRESHOLD"] = "0.001"

`toFactor` is the number to factor. (You may change it.)

In [2]:
toFactor = 16383

The entire algorithm is implemented below:

In [3]:
def MCU_multi(sim, c, k):
    for _ in range(2**k):
        sim.mcmuln(sim, a, [c], m)

In [4]:
%%time
base = int(random.random() * toFactor)
if not gcd(base, toFactor) == 1:
    print("Chose non-relative prime, (without need for quantum computing):")
    print("Factor: {}".format(gcd(base, toFactor)))
else:
    qubitCount = math.ceil(math.log2(toFactor))
    sim = QrackSimulator(2 * qubitCount)

    qi = [i for i in range(qubitCount)]
    qo = [(i + qubitCount) for i in range(qubitCount)]

    # run the quantum subroutine
    for i in qi:
        sim.h(i)
    sim.pown(base, toFactor, qi, qo)
    sim.iqft(qi)
    
    b = [Pauli.PauliZ] * qubitCount
    y = sim.measure_pauli(b, qi)
    r = Fraction(y).limit_denominator(toFactor - 1).denominator

    # try to determine the factors
    if r % 2 != 0:
        r *= 2
    apowrhalf = pow(base, r >> 1, toFactor)
    f1 = gcd(apowrhalf + 1, toFactor)
    f2 = gcd(apowrhalf - 1, toFactor)
    if (not f1 * f2 == toFactor) and f1 * f2 > 1 and int(1.0 * toFactor / (f1 * f2)) * f1 * f2 == toFactor:
        f1, f2 = f1 * f2, int(toFactor / (f1 * f2))
    if f1 * f2 == toFactor and f1 > 1 and f2 > 1:
        print("Factors found : {} * {} = {}".format(f1, f2, toFactor))
    else:
        print("Failed: Found {} and {}".format(f1, f2))

Device #0, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_Intel(R)_Gen9_HD_Graphics_NEO.ir
Device #1, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_NVIDIA_GeForce_RTX_3080_Laptop_GPU.ir
Factors found : 3 * 5461 = 16383
CPU times: user 8.96 s, sys: 63 ms, total: 9.02 s
Wall time: 10.7 s


In [5]:
%%time
base = int(random.random() * toFactor)
if not gcd(base, toFactor) == 1:
    print("Chose non-relative prime, (without need for quantum computing):")
    print("Factor: {}".format(gcd(base, toFactor)))
else:
    qubitCount = math.ceil(math.log2(toFactor))
    sim = QrackSimulator(3 * qubitCount)

    qi = [i for i in range(qubitCount)]
    qo = [(i + qubitCount) for i in range(qubitCount)]
    qa = [(i + 2 * qubitCount) for i in range(qubitCount)]

    # run the quantum subroutine
    for i in qi:
        sim.h(i)
        sim.mcmuln(1<<i, [i], 1<<qubitCount, qo, qa)
        for o in range(len(qa)):
            sim.swap(qa[o], qo[o])
        sim.mcdivn(1<<i, [i], 1<<qubitCount, qo, qa)
        
    sim.iqft(qi)
    
    b = [Pauli.PauliZ] * qubitCount
    y = sim.measure_pauli(b, qi)
    r = Fraction(y).limit_denominator(toFactor - 1).denominator

    # try to determine the factors
    if r % 2 != 0:
        r *= 2
    apowrhalf = pow(base, r >> 1, toFactor)
    f1 = gcd(apowrhalf + 1, toFactor)
    f2 = gcd(apowrhalf - 1, toFactor)
    if (not f1 * f2 == toFactor) and f1 * f2 > 1 and int(1.0 * toFactor / (f1 * f2)) * f1 * f2 == toFactor:
        f1, f2 = f1 * f2, int(toFactor / (f1 * f2))
    if f1 * f2 == toFactor and f1 > 1 and f2 > 1:
        print("Factors found : {} * {} = {}".format(f1, f2, toFactor))
    else:
        print("Failed: Found {} and {}".format(f1, f2))

Factors found : 3 * 5461 = 16383
CPU times: user 1.29 ms, sys: 4.03 ms, total: 5.32 ms
Wall time: 4.5 ms


## Single-control version (Beauregard)

In [6]:
def cua(sim, control, multiplicand, inputs, outputs):
    sim.mcmuln(multiplicand, 1 << len(outputs), [control], inputs, outputs)
    for i in range(len(inputs)):
        sim.cswap([control], inputs[i], outputs[i])
    sim.mcdivn(multiplicand, 1 << len(outputs), [control], inputs, outputs)
    for i in range(len(inputs)):
        sim.m(outputs[i])